In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import os
import nltk
nltk.download('punkt')

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
train_file = pd.read_csv("/content/drive/Shared drives/EMNLP/full_prepped_training.csv")
test_file = pd.read_csv("/content/drive/Shared drives/EMNLP/prepped_test.csv")

In [0]:
# original; edited_sentence; word_position; cosine_similarity
def dataset_gathering(dataset):
    # origin = dataset["original"]
    edited = dataset["edited_sentence"]
    labels = dataset["meanGrade"]
    # position = dataset["word_position"]
    # cosine = dataset["cosine_similarity"]

    res = []
    for sample in zip(edited, labels):
        res.append(sample)
    return res

In [0]:
train_file = dataset_gathering(train_file)
test_file = dataset_gathering(test_file)

In [7]:
print(train_file[0])
print(test_file[0])

('France is ‘ hunting down its citizens who joined twins ’ without trial in Iraq', 0.2)
('The Latest : Election tally shows Cars turning right', 1.2)


In [8]:
print('Tokenizeing...')
print('\n')

def tokenize(data):
    res = []
    for samples in data:
        # nltk.word_tokenize用于取tokens
        temp_t = nltk.word_tokenize(samples[0])
        res.append([temp_t, samples[1]])
    return res


train_file = tokenize(train_file)
test_file = tokenize(test_file)
print(train_file[0])
print(test_file[0])
print('\n')
print('Tokenization completed!')

Tokenizeing...


[['France', 'is', '‘', 'hunting', 'down', 'its', 'citizens', 'who', 'joined', 'twins', '’', 'without', 'trial', 'in', 'Iraq'], 0.2]
[['The', 'Latest', ':', 'Election', 'tally', 'shows', 'Cars', 'turning', 'right'], 1.2]


Tokenization completed!


In [9]:
def lower_case(data):
  print('-----Converting to lowercases-----')
  res = []
  temp_words = []
  for samples in data:
    for words in samples[0]:
      temp_words.append(words.lower())
    res.append([temp_words, samples[1]])
    temp_words = []
  print('-----Complete!-----')
  return res

train_file = lower_case(train_file)
test_file = lower_case(test_file)
print(train_file[0])
print(test_file[0])

-----Converting to lowercases-----
-----Complete!-----
-----Converting to lowercases-----
-----Complete!-----
[['france', 'is', '‘', 'hunting', 'down', 'its', 'citizens', 'who', 'joined', 'twins', '’', 'without', 'trial', 'in', 'iraq'], 0.2]
[['the', 'latest', ':', 'election', 'tally', 'shows', 'cars', 'turning', 'right'], 1.2]


In [0]:
# cacluating maxlen
def maxlen_cal(data):
  print('-----Cacluating maxlen-----')
  maxlen = 0
  for samples in data:
    temp = len(samples[0])
    if temp >= maxlen:
      maxlen = temp
  print('-----Complete!-----')
  return maxlen

In [11]:
print(maxlen_cal(train_file))
print(maxlen_cal(test_file))

-----Cacluating maxlen-----
-----Complete!-----
29
-----Cacluating maxlen-----
-----Complete!-----
27


In [0]:
maxlen = 30

In [0]:
def padding_len(data, maxlen):
  print('-----Padding for dataset-----')
  maxlen = maxlen
  res = []
  for samples in data:
    temp_len = len(samples[0])
    for k in range(maxlen - temp_len):
      samples[0].append('.')
    res.append([samples[0], samples[1]])
  print('-----Complete!-----')
  return res

In [14]:
train_file = padding_len(train_file, maxlen)
test_file = padding_len(test_file, maxlen)

print(maxlen_cal(train_file))
print(maxlen_cal(test_file))

-----Padding for dataset-----
-----Complete!-----
-----Padding for dataset-----
-----Complete!-----
-----Cacluating maxlen-----
-----Complete!-----
30
-----Cacluating maxlen-----
-----Complete!-----
30


In [16]:
print(train_file[0])
print(test_file[0])

[['france', 'is', '‘', 'hunting', 'down', 'its', 'citizens', 'who', 'joined', 'twins', '’', 'without', 'trial', 'in', 'iraq', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 0.2]
[['the', 'latest', ':', 'election', 'tally', 'shows', 'cars', 'turning', 'right', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.'], 1.2]


In [0]:
def corpus_labels_gathering(data):
  corpus_res = []
  labels_res = []
  for samples in data:
    corpus_res.append(samples[0])
    labels_res.append(samples[1])
    
  labels_res = np.array(labels_res)
  return corpus_res, labels_res

In [18]:
x_train, y_train = corpus_labels_gathering(train_file)
x_test, y_test = corpus_labels_gathering(test_file)

print(y_train.shape)
print(y_test.shape)
print(x_train[0])
print(x_test[0])

(17900,)
(3024,)
['france', 'is', '‘', 'hunting', 'down', 'its', 'citizens', 'who', 'joined', 'twins', '’', 'without', 'trial', 'in', 'iraq', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['the', 'latest', ':', 'election', 'tally', 'shows', 'cars', 'turning', 'right', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']


In [19]:
glove_dir = os.path.join('/content/drive/', 'My Drive')
embedding_dim = 300
# Dictionary where we store the word:vector_embedding map
embeddings_index = {}
word_index = {}
count=0

# Setting up embedding array
f = open(os.path.join(glove_dir, 'glove.42B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float64')
    if coefs.shape[0] != embedding_dim:
      continue
    # Embeddings is a dictionary of words:word_vector_embeddings
    embeddings_index[word] = coefs
    word_index[word] = count
    count+=1
f.close()
print('Found {} word vectors.'.format(len(embeddings_index)))

Found 1917494 word vectors.


In [20]:
print('Our word index dictionary is given by: (word, index), a sample 10 entries are:')
list(word_index.items())[:10]

Our word index dictionary is given by: (word, index), a sample 10 entries are:


[(',', 0),
 ('the', 1),
 ('.', 2),
 ('and', 3),
 ('to', 4),
 ('of', 5),
 ('a', 6),
 ('in', 7),
 ('"', 8),
 ('is', 9)]

In [0]:
oov = "OOV"

def oov_vec_gathering(embedding_dim):
  res = np.random.rand(1, embedding_dim)
  return res

oov_vec = oov_vec_gathering(embedding_dim = embedding_dim)

In [22]:
embeddings_index[oov] = oov_vec
word_index[oov] = count + 1
count += 1
print('Found {} word vectors.'.format(len(embeddings_index)))

Found 1917495 word vectors.


In [0]:
# --- Preprocessing the GloVe word-embeddings matrix --
max_words = count
# Instantiating a 10000 x 100 matrix 
# embedding_matrix = (number_words, dim_embedding)
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    # Make sure that we are not exceeding the max token size
    if i < max_words:
        # Get the embedded vector for the word
        embedding_vector = embeddings_index.get(word)
        # Provided that a word is known store it in the 
        # embeddig matrix at position i
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [24]:
print("The size of the word embedding matrix is:" + str(embedding_matrix.shape))

The size of the word embedding matrix is:(1917495, 300)


In [0]:
def data2vec(data):
  res = []
  for samples in data:
    for items in samples:
      if items not in embeddings_index.keys():
        word_index[items] = count
      res.append(word_index[items])

  res = np.array(res)
  return res

In [0]:
x_train = data2vec(x_train)
x_test = data2vec(x_test)

In [0]:
x_train = x_train.reshape(-1, maxlen)
x_test = x_test.reshape(-1, maxlen)

In [0]:
model = keras.models.Sequential([
        keras.layers.Embedding(max_words, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False),
        # keras.layers.Bidirectional(keras.layers.LSTM(2048, activation='tanh', recurrent_activation='sigmoid', use_bias=True, return_sequences=True)),
        # keras.layers.Bidirectional(keras.layers.LSTM(1024, activation='tanh', recurrent_activation='sigmoid', use_bias=True, return_sequences=False)),
        keras.layers.LSTM(2048, activation='tanh', recurrent_activation='sigmoid', use_bias=True, return_sequences=True),
        keras.layers.Bidirectional(keras.layers.LSTM(2048, activation='tanh', recurrent_activation='sigmoid', use_bias=True, return_sequences=False)),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(32, activation='relu'), 
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='relu')                     
])

In [29]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           575248500 
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 2048)          19243008  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 4096)              67125248  
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              4195328   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [0]:
opt = keras.optimizers.Adam(learning_rate=3e-4)

In [0]:
model.compile(optimizer=opt,
              loss = "mean_squared_error",
              metrics = ['mse'])

In [32]:
history = model.fit(x_train, y_train,
            epochs = 10,
            batch_size = 48,
            validation_data = [x_test, y_test])

Train on 17900 samples, validate on 3024 samples
Epoch 1/10
17900/17900 [==============================] - 111s 6ms/step - loss: 0.6254 - mse: 0.6254 - val_loss: 0.4150 - val_mse: 0.4150
Epoch 2/10
17900/17900 [==============================] - 108s 6ms/step - loss: 0.4685 - mse: 0.4685 - val_loss: 0.3367 - val_mse: 0.3367
Epoch 3/10
17900/17900 [==============================] - 108s 6ms/step - loss: 0.4299 - mse: 0.4299 - val_loss: 0.3361 - val_mse: 0.3361
Epoch 4/10
17900/17900 [==============================] - 107s 6ms/step - loss: 0.4004 - mse: 0.4004 - val_loss: 0.3150 - val_mse: 0.3150
Epoch 5/10
17900/17900 [==============================] - 107s 6ms/step - loss: 0.3805 - mse: 0.3805 - val_loss: 0.3136 - val_mse: 0.3136
Epoch 6/10
17900/17900 [==============================] - 107s 6ms/step - loss: 0.3710 - mse: 0.3710 - val_loss: 0.3164 - val_mse: 0.3164
Epoch 7/10
17900/17900 [==============================] - 107s 6ms/step - loss: 0.3535 - mse: 0.3535 - val_loss: 0.3126 - v

In [0]:
predictions_NN = model.predict(x_test)

In [0]:
train_file = pd.read_csv("/content/drive/Shared drives/EMNLP/full_prepped_training.csv")
test_file = pd.read_csv("/content/drive/Shared drives/EMNLP/prepped_test.csv")

In [0]:
def data_gathering_sml(dataset):
    # word_position; euclidean_dist; cosine_similarity; meanGrade
    pos = dataset["word_position"]
    dist = dataset["euclidean_dist"]
    cosine = dataset["cosine_similarity"]
    labels = dataset["meanGrade"]

    res = []
    for sample in zip(pos, dist, cosine, labels):
        res.append(sample)
    
    return res

In [0]:
train_file = data_gathering_sml(train_file)
test_file = data_gathering_sml(test_file)

In [0]:
def attr_labels_gathering(data):
  attr_res = []
  labels_res = []
  for samples in data:
    attr_res.append(samples[:3])
    labels_res.append(samples[-1])
    
  labels_res = np.array(labels_res)
  return attr_res, labels_res

In [0]:
x_train_, y_train_ = attr_labels_gathering(train_file)
x_test_, y_test_ = attr_labels_gathering(test_file)

In [0]:
from sklearn import linear_model
from sklearn import svm
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [40]:
svm_ = svm.SVR()
svm_.fit(x_train_, y_train_)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [41]:
predictions_svm = svm_.predict(x_test_)
print(mean_squared_error(y_test_, predictions_svm))

0.3446791963760914


In [42]:
ridge_ = linear_model.Ridge(alpha=0.5)
ridge_.fit(x_train_, y_train_)

Ridge(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [43]:
predictions_ridge = ridge_.predict(x_test_)
print(mean_squared_error(y_test_, predictions_ridge))

0.34564278233732865


In [44]:
print(len(predictions_svm))
print(len(predictions_ridge))
print(len(predictions_NN))

3024
3024
3024


In [45]:
pred_list = []
for i in range(len(predictions_NN)):
    temp = [np.float(predictions_NN[i]), predictions_ridge[i], predictions_svm[i]]
    pred_list.append(temp)

print(pred_list[:5])
print(len(pred_list))

[[0.9513710737228394, 1.0949618021581715, 1.1876257231346061], [1.0973364114761353, 1.1347363731653313, 1.1888836220901866], [0.837093710899353, 1.2205052828803515, 1.24569218777683], [0.8677194714546204, 1.2233738387397715, 1.2107516530255127], [0.8231667280197144, 1.011845855375808, 0.9932851824619386]]
3024


In [0]:
def predictions_fusion(predictions_list, method="mean"):
    res = []
    for sublist in predictions_list:

        if method == "mean":
            temp = np.mean(sublist)
            res.append([temp])
        
        if method == "min":
            temp = min(sublist)
            res.append([temp])
        
        if method == "max":
            temp = max(sublist)
            res.append([temp])
    
    return np.array(res)

In [0]:
mean_prediction = predictions_fusion(pred_list, method="mean")
min_prediction = predictions_fusion(pred_list, method="min")
max_prediction = predictions_fusion(pred_list, method="max")

In [55]:
print(np.sqrt(mean_squared_error(y_test_, mean_prediction)))
print(np.sqrt(mean_squared_error(y_test_, min_prediction)))
print(np.sqrt(mean_squared_error(y_test_, max_prediction)))

0.5680877897876262
0.5538813657680356
0.5948010963482846
